In [1]:
import pandas as pd
from zipfile import ZipFile
from  pathlib import Path
import calendar
import datetime
import pickle

In [2]:
example = pd.read_excel(r'.\codes\Example.xlsx', sheet_name = 'example')


In [5]:
example.columns


Index(['product_lvls', 'category', 'demo_lvls', 'socdem_gr',
       'time_period_type', 'year', 'period_lbl', 'buying_households',
       'raw_shoppers', 'loyalty_volume_based',
       'loyalty_value_based_local_currency', 'loyalty_value_based_usd',
       'percent_2__time_buyers', 'percent_household_penetration',
       'purchase_frequency', 'occasions', 'item_buying_rate_local_currency',
       'item_buying_rate_usd', 'purchase_size_local_currency',
       'purchase_size_usd', 'purchase_size_su',
       'average_per_su_local_currency', 'average_per_su_usd', 'volume_su',
       'volume_physical_units', 'spend_local_currency', 'spend_usd',
       'product_hier', 'demo_hier', 'products', 'features', 'cat_segment',
       'demo_groups'],
      dtype='object')

In [48]:
path = Path(r'C:\Users\nyzhur\Git\pg-dt-portal-for-nz\codes\periods.xlsx')

In [2]:
new_data =pd.read_parquet('data/new_data_merged/new_data.pq')
col_check = {}
for col in new_data.columns:
    if new_data[col].isna().sum():
        col_check[col] = new_data[col].isna().sum()
print(col_check)
      

{}


In [3]:
meta_information = new_data[['category','period_lbl', 'demo_groups','shop_code']].drop_duplicates()
meta_information = meta_information.sort_values(by = ['category','period_lbl', 'demo_groups','shop_code']).reset_index(drop=True)



In [5]:
new_data.columns

Index(['spend_local_currency', 'volume_su', 'volume_physical_units',
       'occasions', 'buying_households', 'raw_shoppers',
       'percent_household_penetration', 'purchase_frequency',
       'item_buying_rate_local_currency', 'purchase_size_local_currency',
       'purchase_size_su', 'average_per_su_local_currency', 'year', 'features',
       'category', 'products', 'product_hier', 'product_lvls', 'demo_groups',
       'demo_hier', 'demo_lvls', 'socdem_gr', 'cat_segment', 'shop_code',
       'shop_lvls', 'channel_code', 'file', 'period_lbl', 'time_period_type'],
      dtype='object')

In [55]:
meta_2 = new_data.groupby(['category','period_lbl', 'demo_groups','shop_code'])['file'].agg(set)

In [62]:
meta_2['set'].map(len).value_counts()

set
1    19941
2       84
Name: count, dtype: int64

In [61]:
meta_2[meta_2['set'].map(len)==2]

set  \
category period_lbl demo_groups shop_code                                                      
1        108        1           1          {out_datatile/diapers_socdem-new24.parquet, ou...   
         109        1           1          {out_datatile/diapers_socdem-new24.parquet, ou...   
         117        1           1          {out-for-datatile-sep24/diapers_allshops.parqu...   
         119        1           1          {out-for-datatile-sep24/diapers_allshops.parqu...   
2        107        1           1          {out-for-datatile/br_allshops.parquet, out_dat...   
...                                                                                      ...   
6        115        1           1          {out-for-datatile-sep24/deterg_allshops.parque...   
         116        1           1          {out-for-datatile-sep24/deterg_allshops.parque...   
         117        1           1          {out-for-datatile-sep24/deterg_allshops.parque...   
         119        1           1          {out-for-datatile-sep24/deterg_allshops.parque...   
         120        1           1          {out-for-datatile-sep24/deterg_allshops.parque...   

                                            len  
category period_lbl demo_groups shop_code        
1        108        1           1           650  
         109        1           1           618  
         117        1           1           654  
         119        1           1           620  
2        107        1           1           684  
...                                         ...  
6        115        1           1          2614  
         116        1           1          2484  
         117        1           1          2618  
         119        1           1          2510  
         120        1           1          2396  

[84 rows x 2 columns]

In [20]:
meta_2 = pd.DataFrame(meta_2)

In [30]:
meta_2['len'] = meta_2['file'].str.(',')

In [45]:
meta_2.loc[1]

file  \
period_lbl demo_groups shop_code                                                      
9          1           1          [out-for-datatile/diapers_allshops.parquet, ou...   
                       2          [out-for-datatile/diapers_allshops.parquet, ou...   
                       4          [out-for-datatile/diapers_allshops.parquet, ou...   
                       5          [out-for-datatile/diapers_allshops.parquet, ou...   
                       8          [out-for-datatile/diapers_allshops.parquet, ou...   
...                                                                             ...   
119        49          1          [out_datatile/diapers_socdem-new24.parquet, ou...   
           50          1          [out_datatile/diapers_socdem-new24.parquet, ou...   
           51          1          [out_datatile/diapers_socdem-new24.parquet, ou...   
           52          1          [out_datatile/diapers_socdem-new24.parquet, ou...   
           53          1          [out_datatile/diapers_socdem-new24.parquet, ou...   

                                  len  
period_lbl demo_groups shop_code       
9          1           1          NaN  
                       2          NaN  
                       4          NaN  
                       5          NaN  
                       8          NaN  
...                               ...  
119        49          1          NaN  
           50          1          NaN  
           51          1          NaN  
           52          1          NaN  
           53          1          NaN  

[986 rows x 2 columns]

In [73]:
with open('data/tmp/old_data.pickle', 'rb') as f:
    old_data = pickle.load(f)

In [74]:
df_union = pd.concat([old_data, new_data], ignore_index=True)

In [65]:
i = df_union.shape
i

(13857113, 37)

In [66]:
df_union_direct_drop = df_union.drop_duplicates()

In [67]:
df_union_direct_drop.shape

(13857113, 37)

In [68]:
example = pd.read_excel(r'.\codes\Example.xlsx', sheet_name = 'example')
fin_cols = example.columns.to_list() + ['shop_code', 'shop_lvls','channel_code']


In [69]:
df_union_list_drop = df_union.drop_duplicates(subset = fin_cols )

In [71]:
i[0] -df_union_list_drop.shape[0]

1985

In [72]:
fin_cols

['product_lvls',
 'category',
 'demo_lvls',
 'socdem_gr',
 'time_period_type',
 'year',
 'period_lbl',
 'buying_households',
 'raw_shoppers',
 'loyalty_volume_based',
 'loyalty_value_based_local_currency',
 'loyalty_value_based_usd',
 'percent_2__time_buyers',
 'percent_household_penetration',
 'purchase_frequency',
 'occasions',
 'item_buying_rate_local_currency',
 'item_buying_rate_usd',
 'purchase_size_local_currency',
 'purchase_size_usd',
 'purchase_size_su',
 'average_per_su_local_currency',
 'average_per_su_usd',
 'volume_su',
 'volume_physical_units',
 'spend_local_currency',
 'spend_usd',
 'product_hier',
 'demo_hier',
 'products',
 'features',
 'cat_segment',
 'demo_groups',
 'shop_code',
 'shop_lvls',
 'channel_code']

In [79]:
meta_2 = df_union.groupby(['cat_segment','category','period_lbl', 'demo_groups','shop_code'])['file'].agg(set)

In [87]:
meta_2.map(len).value_counts()

file
1    169624
2      2257
3         2
Name: count, dtype: int64

In [86]:
meta_2.loc[meta_2.map(len)>1]

cat_segment  category  period_lbl  demo_groups  shop_code
100          1         9           1            1            {PG_Flatfiles_Diapers, out-for-datatile/diaper...
                       12          1            1            {PG_Flatfiles_Diapers, out-for-datatile/diaper...
                       15          1            1            {PG_Flatfiles_Diapers, out-for-datatile/diaper...
                       18          1            1            {PG_Flatfiles_Diapers, out-for-datatile/diaper...
                       21          1            1            {PG_Flatfiles_Diapers, out-for-datatile/diaper...
                                                                                   ...                        
611          6         115         1            1            {out-for-datatile-sep24/deterg_allshops.parque...
                       116         1            1            {out-for-datatile-sep24/deterg_allshops.parque...
                       117         1            1     